In [ ]:
# 这是一个用来得到ec的精细话数据的脚本

# 数据类型
1. vis 
2. uv_10
3. tmax
4. tmin
5. tcc 
6. t2
7. skt
8. lsp(大尺度降水)
9. conv(对流降水)


In [2]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
import netCDF4 
import h5py     
# 测试数据的可用性
def read_data():
        file_path = "/home/liyuan3970/Data/My_Git/2022041700/"        
        file_name = file_path +"ecfine.I2022041700.000.F2022041700.nc"
        f = netCDF4.Dataset(file_name)
        data_xr_nc = f.variables['t']
        lat = f.variables['latP'][:]
        lon = f.variables['lonP'][:]
        time = f.variables['time'][:]
          
        # return lat,lon,time,data_xr_nc
read_data()


In [23]:
# 面向过程的编程
import xarray as xr
file_path = "/home/liyuan3970/Data/My_Git/2022041700/*.nc" 
f = xr.open_mfdataset(file_path, parallel=True)




# 数据的存储

In [90]:
# 单数据的完整存储
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
import xesmf as xe
import redis
import xarray as xr
file_path = "/home/liyuan3970/Data/My_Git/2022041700/*.nc" 
f = xr.open_mfdataset(file_path, parallel=True)

vis=f.vis.sel(lev=1000,lonS=slice(120,122),latS=slice(29,27.5))
vis =  vis.swap_dims({'latS':'lat','lonS':'lon'})
ds_out = xr.Dataset(
            {   
                
                "lat": (["lat"], np.arange(27.8, 29.5, 0.05)),
                "lon": (["lon"], np.arange(120, 122, 0.05)),
            }
        )
regridder = xe.Regridder(vis, ds_out, "bilinear")
# vis = vis.reset_coords(names=['time','lat','lon'], drop=True)
dr_out = regridder(vis)
# vis
data = dr_out.sel(lat = 28.5,lon = 121.5,method='nearest').to_pandas()
a = data.tolist()
a

/opt/conda/lib/python3.7/site-packages/dask/array/core.py:343: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)


[nan,
 1750.00341796875,
 1814.71630859375,
 1748.47705078125,
 1679.7958984375,
 1674.606689453125,
 1583.64208984375,
 1584.863037109375,
 1623.93505859375,
 1710.626220703125,
 1709.405029296875,
 1677.96435546875,
 1591.2734375,
 1443.532470703125,
 1188.0380859375,
 1272.897705078125,
 1598.599365234375,
 1721.92041015625,
 1793.95947265625,
 1688.953369140625,
 1663.61767578125,
 1611.72509765625,
 1513.739990234375,
 1384.31396484375,
 1696.279296875,
 1798.5380859375,
 1605.6201171875,
 661.787109375,
 1363.251708984375,
 1668.806884765625,
 1205.132080078125,
 1086.695068359375,
 1397.744873046875,
 1645.302734375,
 1210.626708984375,
 911.176513671875,
 1521.37109375,
 1231.68896484375,
 1218.563232421875,
 571.43310546875,
 816.548828125,
 1220.39453125,
 871.1884765625,
 1196.890380859375,
 1402.93408203125,
 1447.1953125,
 962.45849609375,
 1122.104248046875,
 1070.822021484375,
 1434.68017578125,
 1093.41064453125,
 896.21923828125,
 266.48828125]

In [24]:
f

,Array,Chunk
Bytes,1.10 GB,20.81 MB
Shape,"(15, 53, 481, 721)","(15, 1, 481, 721)"
Count,281 Tasks,53 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,1.10 GB,20.81 MB
Shape,"(15, 53, 481, 721)","(15, 1, 481, 721)"
Count,281 Tasks,53 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [26]:
# 读取固定点的数据
#选择固定经纬度范围的数据
# sub=ff.sel(lon=slice(105,120),lat=slice(15,30))
vis=f.vis.sel(lev=1000,lonS=slice(120,122),latS=slice(29,27.5))
# vis=f.vis
# t =f.t
# t.sel(lev=1000,lonP=125,latP=27.5,method='nearest').data
vis


,Array,Chunk
Bytes,702.78 kB,13.26 kB
Shape,"(15, 53, 13, 17)","(15, 1, 13, 17)"
Count,334 Tasks,53 Chunks
Type,float32,numpy.ndarray


In [18]:
# 线性插值的核心
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import interp1d

x = [1,2,3,4,5,6,7,8,9,10]
y = [1,2,3,4,5,6,7,8,9,10] # 对应没每个点的值
f1=interp1d(x,y,kind='linear')#,kind='cubic'
f1=interp1d(x,y,kind='cubic')#,kind='cubic'
x_pre = [1.1,2.2,3.3,4.4,5.5,6.6,7.7,8.8,9.9]

y_pre = f1(x_pre)

y_pre


array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [21]:
# 数据结构的设计
data = {
    '淡竹':{
        'value1':[53],
        'value2':[53],
        'value3':[53]
    },
    '洪家':{
        'value1':[53],
        'value2':[53],
        'value3':[53]
    }
    }

In [35]:
import numpy as np
# for i in range(1,53*3,3):
#     print(i)

x_da = [x for x in range(1,53*3+1,3) ]
print(len(x_da))

53


In [91]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
import xesmf as xe
import redis

class ec_data_point:
    def __init__(self, select_time,select_type,select_lat,select_lon): 
        self.var_list = ['vis','u10','v10','tmax2','tmin2','tcc','t2','skt','lsp','cp']
        self.county_name = ['station1']
        self.county_lat = [29.0]
        self.county_lon = [121.2]
        self.data = { }
        self.read_data()
    def regrid_data(self,data):
        ds_out = xr.Dataset(
            {   
                
                "lat": (["lat"], np.arange(27.8, 29.5, 0.05)),
                "lon": (["lon"], np.arange(120, 122, 0.05)),
            }
        )
        regridder = xe.Regridder(data, ds_out, "bilinear")
        dr_out = regridder(data)
        return dr_out
    def read_data(self):
        file_path = "/home/liyuan3970/Data/My_Git/2022041700/*.nc" 
        f = xr.open_mfdataset(file_path, parallel=False)
        # 列表数据--
        vis = f.vis.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        vis =  vis.swap_dims({'latS':'lat','lonS':'lon'})
        u10 = f.u10.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        u10 =  u10.swap_dims({'latS':'lat','lonS':'lon'})
        v10 = f.v10.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        v10 =  v10.swap_dims({'latS':'lat','lonS':'lon'})
        tmax2 = f.tmax2.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        tmax2 =  tmax2.swap_dims({'latS':'lat','lonS':'lon'})
        tmin2 = f.tmin2.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        tmin2 =  tmin2.swap_dims({'latS':'lat','lonS':'lon'})
        tcc = f.tcc.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        tcc =  tcc.swap_dims({'latS':'lat','lonS':'lon'})
        t2 = f.t2.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        t2 =  t2.swap_dims({'latS':'lat','lonS':'lon'})
        skt = f.skt.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        skt =  skt.swap_dims({'latS':'lat','lonS':'lon'})
        lsp = f.lsp.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8))
        lsp =  lsp.swap_dims({'latS':'lat','lonS':'lon'})
        cp = f.cp.sel(lev=1000,lonS=slice(120,122),latS=slice(29.5,27.8)) 
        cp =  cp.swap_dims({'latS':'lat','lonS':'lon'})        
        # 空间插值----------------------------------
        # ----------------------------------------
        # ----------------------------------------
        # 此处为空间插值函数0.05
        grid_vis = self.regrid_data(vis)
        grid_u10 = self.regrid_data(u10)
        grid_v10 = self.regrid_data(v10)
        grid_tmax2 = self.regrid_data(tmax2)
        grid_tmin2 = self.regrid_data(tmin2)
        grid_tcc = self.regrid_data(tcc)
        grid_t2 = self.regrid_data(t2)
        grid_skt = self.regrid_data(skt)
        grid_lsp = self.regrid_data(lsp)
        grid_cp = self.regrid_data(cp)
        # 数据的重新加载
        self.data['vis'] = grid_vis
        self.data['u10'] = grid_u10
        self.data['v10'] = grid_v10
        self.data['tmax2'] = grid_tmax2
        self.data['tmin2'] = grid_tmin2
        self.data['tcc'] = grid_tcc
        self.data['t2'] = grid_t2
        self.data['skt'] = grid_skt
        self.data['lsp'] = grid_lsp
        self.data['cp'] = grid_cp    
        # print(grid_vis.sel(lon=121.5, lat=28.5,method='nearest'))  
    def get_data(self):
        # get_point_data ={
        #     'station1':{
        #         'vis':vis
        #     }
        # }
        get_point_data ={}
        for i in range(len(self.county_lat)):
            get_point_data[str(self.county_name[i])] = {}
            for var in self.var_list:
                select_data = self.data[var]
                # 就近读取数据
                selelct_point_data =  select_data.sel(lon=self.county_lon[i], lat=self.county_lat[i],method='nearest')
                # 时间插值
                self.interp1d_data(selelct_point_data)
                get_point_data[str(self.county_name[i])][var] = selelct_point_data
        return get_point_data                     
    def get_single(self,select_lat,select_lon):
        '''用于处理特定经纬度数据'''
        # single_point_data = {
        #     'vis':vis
        # }
        single_point_data = {}
        for var in self.var_list:
            select_data = self.data[var]
            single_point_data[var] = select_data.sel(lon=select_lon, lat=select_lat,method='nearest').to_pandas().tolist()
        return single_point_data                
    def interp1d_data(self,point_data):
        ''' 将get_data和get_single_data的数据进行时间插值'''
        print("数据长度:",len(point_data[0,:]),point_data)
        num = len(point_data[0,:])
        x_da = [x for x in range(1,num*3,3)]
        x_pre = [x for x in range(1,num*3-3,1)]
        y_da = point_data[0,:].data
        f_func=interp1d(x_da,y_da,kind='cubic')
        y_pre = f_func(x_pre)
        return y_pre
    def decode_latlon(self):
        pass
    def puton_redis(self):
        host = '192.168.192.5' # redis服务地址
        port = 6379  # redis服务端口
        # 连接数据库
        r = redis.StrictRedis(host=host,port=port, password="lq9394",db=0)
        value = str(self.data)
        name = 'EC_data'
        # ex - 过期时间（秒）
        # px - 过期时间（毫秒）
        # nx - 如果设置为True，则只有name不存在时，当前set操作才执行
        # xx - 如果设置为True，则只有name存在时，当前set操作才执行
        r.set(name, value, ex=60*60*24, px=None, nx=False, xx=False)

        

        

select_time,select_type,select_lat,select_lon = '2022041700','t',27.5,125.7
ec_worker = ec_data_point(select_time,select_type,select_lat,select_lon)       
# regrid_data = ec_worker.regrid_data()
# read_data = ec_worker.read_data()


/opt/conda/lib/python3.7/site-packages/dask/array/core.py:343: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)


In [92]:
# ec_worker.puton_redis()

ec_worker.get_single(28.5,121.5)

{'vis': [nan,
  1750.00341796875,
  1814.71630859375,
  1748.47705078125,
  1679.7958984375,
  1674.606689453125,
  1583.64208984375,
  1584.863037109375,
  1623.93505859375,
  1710.626220703125,
  1709.405029296875,
  1677.96435546875,
  1591.2734375,
  1443.532470703125,
  1188.0380859375,
  1272.897705078125,
  1598.599365234375,
  1721.92041015625,
  1793.95947265625,
  1688.953369140625,
  1663.61767578125,
  1611.72509765625,
  1513.739990234375,
  1384.31396484375,
  1696.279296875,
  1798.5380859375,
  1605.6201171875,
  661.787109375,
  1363.251708984375,
  1668.806884765625,
  1205.132080078125,
  1086.695068359375,
  1397.744873046875,
  1645.302734375,
  1210.626708984375,
  911.176513671875,
  1521.37109375,
  1231.68896484375,
  1218.563232421875,
  571.43310546875,
  816.548828125,
  1220.39453125,
  871.1884765625,
  1196.890380859375,
  1402.93408203125,
  1447.1953125,
  962.45849609375,
  1122.104248046875,
  1070.822021484375,
  1434.68017578125,
  1093.41064453125,

In [41]:
data = ec_worker.data['vis']
data
#.sel(lonS=select_lon, latS=select_lat,lev = 1000,method='nearest')


,Array,Chunk
Bytes,4.32 MB,81.60 kB
Shape,"(15, 53, 34, 40)","(15, 1, 34, 40)"
Count,389 Tasks,53 Chunks
Type,float32,numpy.ndarray


In [56]:

data[0,1,21,1].data

dask.array<getitem, shape=(), dtype=float32, chunksize=()>